# Try to understand how tridentNet was built with detectron2

## TridentNet/train_net.py:

**default_argument_parser:**
- Create a parser with some common arguments used by detectron2 users.
- Returns: argparse.ArgumentParser
- `import argparse`: the recommended command-line parsing module in the Python standard library.

In [1]:
from detectron2.engine import default_argument_parser

In [2]:
??default_argument_parser

Signature: default_argument_parser()
Source:   
def default_argument_parser():
    """
    Create a parser with some common arguments used by detectron2 users.

    Returns:
        argparse.ArgumentParser:
    """
    parser = argparse.ArgumentParser(description="Detectron2 Training")
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "--resume",
        action="store_true",
        help="whether to attempt to resume from the checkpoint directory",
    )
    parser.add_argument("--eval-only", action="store_true", help="perform evaluation only")
    parser.add_argument("--num-gpus", type=int, default=1, help="number of gpus *per machine*")
    parser.add_argument("--num-machines", type=int, default=1)
    parser.add_argument(
        "--machine-rank", type=int, default=0, help="the rank of this machine (unique per machine)"
    )

    # PyTorch still may leave orphan processes in multi-gpu training.
    # Th

***magic commands to handle .py files:***
- 参考[user2304916](https://stackoverflow.com/questions/21034373/how-to-load-edit-run-save-text-files-py-into-an-ipython-notebook-cell)在stackoverflow的回答。
- `%load <filename.py>`: the content of `filename.py` will be loaded in the next cell. You can edit and execute it as usual.
- `%run <filename.py>`: allows you to run any python script and load all of its data directly into the interactive namespace
- `%%writefile <filename.py>`: saves the cell content back into a file. you should add the magic command at the **top** of the cell and run it. Beaware that if a file with the same name already exists it will **be silently overwritten**.
- `%magic`:  general help on magic functions
- `%lsmagic`: a list of the available magic functions
- To see the help for any magic command add a `?`: like `%load?` or `%%writefile?`
- See also: [Magic functions](http://ipython.org/ipython-doc/dev/interactive/tutorial.html#magic-functions) from the official IPython docs.

***Argparse Tutorial:***
- 参考[argparse tutorial](https://docs.python.org/3/howto/argparse.html)和[argparse docs](https://docs.python.org/zh-cn/3/library/argparse.html)

methods:
- `add_argument()`: specify which command-line options the program is willing to accept
    - `'<positional_argument_name>'`: positional argument
    - `'--<optional_argument_name>'`: optional argument
    - `'-<short_option_name>'`: the short type of an optional argument 
    - `type = <type>`: specify the data type of an argument, default to string
    - `help = <contents>`: explain what the argument is when run '--help' or '-h'
    - `action = 'store_true`: if the option is specified, assign the value True to the argument. Not specifying it implies False.
    - `action = 'count'`:  to count the number of occurrences of a specific optional arguments. If you don’t specify the argument, that argument is considered to have None value. If you have a default value, it will add the number of occurrences to it.
    - `choices = [choice1, choice2, ...]`: restrict the values the argument option can accept 
    - `default = <default_value>`: specify the default value
- `parse_args()`: return data from options specified
- `add_mutually_exclusive_group()`: specify options that **conflict with each other**
- `metavar = <metavar_name>`: A name for the argument in usage messages(在使用方法消息中使用的参数值示例, e.g. `metavar='FILE'`, `metavar = 'N'`)

In [3]:
%%writefile ./codes/argParse.py
# %load ./codes/argParse.py
import argparse
parser = argparse.ArgumentParser()
group =  parser.add_mutually_exclusive_group()
# calling this program now need to specify 2 argument 'echo' and 'square'
# Positional arguments
parser.add_argument('square', type = int, help = 'display a square of a given number')
# Optional arguments
group.add_argument('-n','--noise', action='store_true',
                   help='make some noise!')
group.add_argument('-q', '--quiet', action='store_true',
                   help = 'be qiuet!')
parser.add_argument('-v','--verbosity', help = 'increase output verbosity', choices = [1, 2, 3], type = int)
parser.add_argument('-c','--count', default = 100, 
                    help = 'count  the number of occurrences of a specific optional arguments', action = 'count')
args = parser.parse_args()
answer = args.square**2
if args.count:
    print(f'It counts {args.count}')
    print(type(args.count))
if args.verbosity:
    print(f'verbosity {args.verbosity} turned on')
if args.noise:
    print('the square of {} equals {}'.format(args.square, answer))
elif args.quiet:
    print(answer)
else:
    print(f'{answer} !!!')

Overwriting ./codes/argParse.py


In [4]:
%run ./codes/argParse.py  -v 1 -q 4 

It counts 100
<class 'int'>
verbosity 1 turned on
16


In [5]:
%run ./codes/argParse.py --verbosity 2 4  --count --count --count

It counts 103
<class 'int'>
verbosity 2 turned on
16 !!!


In [6]:
%run ./codes/argParse.py 4 

It counts 100
<class 'int'>
16 !!!


In [7]:
%run ./codes/argParse.py --help

usage: argParse.py [-h] [-n | -q] [-v {1,2,3}] [-c] square

positional arguments:
  square                display a square of a given number

optional arguments:
  -h, --help            show this help message and exit
  -n, --noise           make some noise!
  -q, --quiet           be qiuet!
  -v {1,2,3}, --verbosity {1,2,3}
                        increase output verbosity
  -c, --count           count the number of occurrences of a specific optional
                        arguments


In [19]:
%%writefile ./codes/default_argument_parser_detectron2.py

import argparse
import os

def default_argument_parser():
    """
    Create a parser with some common arguments used by detectron2 users.

    Returns:
        argparse.ArgumentParser:
    """
    parser = argparse.ArgumentParser(description='Learning Detectron2:train_net.py')
    parser.add_argument('--config-file', default='', metavar='FILE', help='path to config file')
    parser.add_argument(
        '--resume',
        action='store_true',
        help='whether to attempt to resume from the checkpoint directory'
    )
    parser.add_argument('--eval-only', action='store_true', help='perform evaluation only')
    parser.add_argument('--num-gpus', type=int, default=1, help='number of gpus *per machine*')
    parser.add_argument('--num-machines', type=int, default=1)
    parser.add_argument(
        '--machine-rank', type=int, default=0, help='the rank of this machine (unique per machine)'
        )
    port = 2 ** 15 + 2 ** 14 + hash(os.getuid()) % 2 ** 14
    parser.add_argument('--dist-url', default=f'tcp://127.0.0.1:{port}')
    parser.add_argument(
        'opts',
        help='Modify config options using the command-line',
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser

if __name__ == '__main__':
    args = default_argument_parser().parse_args()

Overwriting ./codes/default_argument_parser_detectron2.py


In [21]:
%run ./codes/default_argument_parser_detectron2.py \
--config-file='projects/TridentNet/configs/tridentnet_fast_R_50_C4_1x.yaml' \
SOLVER.IMS_PER_BATCH 2 SOLVER.BASE_LR 0.0025

In [23]:
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='projects/TridentNet/configs/tridentnet_fast_R_50_C4_1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=['SOLVER.IMS_PER_BATCH', '2', 'SOLVER.BASE_LR', '0.0025'], resume=False)


In [24]:
from detectron2.engine import launch

In [26]:
??launch

Signature: launch(main_func, num_gpus_per_machine, num_machines=1, machine_rank=0, dist_url=None, args=())
Source:   
def launch(main_func, num_gpus_per_machine, num_machines=1, machine_rank=0, dist_url=None, args=()):
    """
    Args:
        main_func: a function that will be called by `main_func(*args)`
        num_machines (int): the total number of machines
        machine_rank (int): the rank of this machine (one per machine)
        dist_url (str): url to connect to for distributed training, including protocol
                       e.g. "tcp://127.0.0.1:8686".
                       Can be set to auto to automatically select a free port on localhost
        args (tuple): arguments passed to main_func
    """
    world_size = num_machines * num_gpus_per_machine
    if world_size > 1:
        # https://github.com/pytorch/pytorch/pull/14391
        # TODO prctl in spawned processes

        if dist_url == "auto":
            assert num_machines == 1, "dist_url=auto cannot work wi

It seems that it just run the main_func if I have just 1 gpu. 
I will study this in detail when I can train a model on the server.